website example. nvm

In [ ]:
from sb3_plus import MultiOutputPPO
from stable_baselines3.common.evaluation import evaluate_policy
import gymnasium as gym
import gymnasium_hybrid
import time

# Create environment
env = gym.make('Moving-v0', render_mode='rgb_array')

# Instantiate the agent
model = MultiOutputPPO(
    policy='MultiOutputPolicy',
    env=env,
    verbose=1,
    policy_kwargs=dict(
        net_arch=dict(pi=[252] * 4, vf=[252] * 4)
    )
)

# Train the agent and display a progress bar
model.learn(
    total_timesteps=int(2e5),
    progress_bar=True
)

# Save the agent
model.save("ppo_moving")
del model  # delete trained model to demonstrate loading

# Load the trained agent
model = MultiOutputPPO.load("ppo_moving", env=env)

# Evaluate the agent
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
print(f'Mean Reward {mean_reward} | Std Reward {std_reward}')

# Enjoy trained agent
vec_env = model.get_env()
obs = vec_env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render('human')

Raw 


In [ ]:
from sb3_plus import MultiOutputPPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch as th
import torch.nn as nn

class UnoFeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=256):
        super(UnoFeatureExtractor, self).__init__(observation_space, features_dim)
        self.layer = nn.Sequential(
            nn.Linear(observation_space.shape[0], features_dim),
            nn.ReLU(),
            nn.Linear(features_dim, features_dim),
            nn.ReLU()
        )
        
    def forward(self, observations):
        return self.layer(observations)

env = UNOCommunicationEnv()  

model = MultiOutputPPO(
    policy="MultiOutputPolicy",
    env=env,
    policy_kwargs=dict(
        features_extractor_class=UnoFeatureExtractor,
        features_extractor_kwargs=dict(features_dim=256),
        net_arch=dict(pi=[256, 256], vf=[256, 256])  
    ),
    verbose=1
)

model.learn(total_timesteps=10000)

model.save("uno_model_based_agent")


Extractor idea


In [ ]:
from sb3_plus import MultiOutputPPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch as th
import torch.nn as nn

class UnoFeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=256, message_dim=8):
        super(UnoFeatureExtractor, self).__init__(observation_space, features_dim)
        
        self.message_dim = message_dim

        # 1. last action + position
        self.action_actor_layer = nn.Sequential(
            nn.Linear(2, 64),  
            nn.ReLU()
        )

        # 2. availiable actions
        self.available_actions_layer = nn.Sequential(
            nn.Linear(62, 64),
            nn.ReLU()
        )
        
        # 3. cards on hand
        self.cards_on_hand_layer = nn.Sequential(
            nn.Linear(108, 64),
            nn.ReLU()
        )
        
        # 4. message + player position
        self.message_layer = nn.Sequential(
            nn.Linear(self.message_dim + 1, 64),
            nn.ReLU()
        )

        # combined
        self.combined_layer = nn.Sequential(
            nn.Linear(64 * 4, features_dim),
            nn.ReLU(),
            nn.Linear(features_dim, features_dim),
            nn.ReLU()
        )

    def forward(self, observations):

        # 1. 
        last_action, actor_position = observations[0][0]
        action_actor_features = self.action_actor_layer(th.tensor([last_action, actor_position]).float())

        # 2. 
        available_actions = th.tensor(observations[0][1]).float()
        available_actions_features = self.available_actions_layer(available_actions)

        # 3.
        cards_on_hand = th.tensor(observations[0][2]).float()
        cards_on_hand_features = self.cards_on_hand_layer(cards_on_hand)

        # 4. 
        messages = observations[0][3]
        message_features = th.stack([
            self.message_layer(th.cat([th.tensor(msg[0]).float(), th.tensor([msg[1]]).float()]))
            for msg in messages
        ]).sum(dim=0) 

        # combine
        combined_features = th.cat([
            action_actor_features, available_actions_features, cards_on_hand_features, message_features
        ], dim=-1)
        
        return self.combined_layer(combined_features)
    
    

env = UNOCommunicationEnv() 

model = MultiOutputPPO(
    policy="MultiOutputPolicy",
    env=env,
    policy_kwargs=dict(
        features_extractor_class=UnoFeatureExtractor,
        features_extractor_kwargs=dict(features_dim=256),
        net_arch=dict(pi=[256, 256], vf=[256, 256])  
    ),
    verbose=1
)

model.learn(total_timesteps=10000)

model.save("uno_based_agent")

step function should return observations, rewards, dones, infos
